In [58]:
#Credits to https://github.com/Britefury

from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import scipy
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [59]:
class BSDSDataset(object):
    """
    BSDS dataset wrapper
    Given the path to the root of the BSDS dataset, this class provides
    methods for loading images, ground truths and evaluating predictions
    Attribtes:
    bsds_path - the root path of the dataset
    data_path - the path of the data directory within the root
    images_path - the path of the images directory within the data dir
    gt_path - the path of the groundTruth directory within the data dir
    train_sample_names - a list of names of training images
    val_sample_names - a list of names of validation images
    test_sample_names - a list of names of test images
    """
    def __init__(self, bsds_path):
        """
        Constructor
        :param bsds_path: the path to the root of the BSDS dataset
        """
        self.bsds_path = bsds_path
        self.data_path = os.path.join(bsds_path, 'BSDS500', 'data')
        self.images_path = os.path.join(self.data_path, 'images')
        self.gt_path = os.path.join(self.data_path, 'groundTruth')

        self.train_sample_names = self._sample_names(self.images_path, 'train')
        self.val_sample_names = self._sample_names(self.images_path, 'val')
        self.test_sample_names = self._sample_names(self.images_path, 'test')

    @staticmethod
    def _sample_names(dir, subset):
        names = []
        files = os.listdir(os.path.join(dir, subset))
        for fn in files:
            dir, filename = os.path.split(fn)
            name, ext = os.path.splitext(filename)
            if ext.lower() == '.jpg':
                names.append(os.path.join(subset, name))
        return names

    def read_image(self, name):
        """
        Load the image identified by the sample name (you can get the names
        from the `train_sample_names`, `val_sample_names` and
        `test_sample_names` attributes)
        :param name: the sample name
        :return: a (H,W,3) array containing the image, scaled to range [0,1]
        """
        path = os.path.join(self.images_path, name + '.jpg')
        return img_as_float(imread(path))

    def get_image_shape(self, name):
        """
        Get the shape of the image identified by the sample name (you can
        get the names from the `train_sample_names`, `val_sample_names` and
        `test_sample_names` attributes)
        :param name: the sample name
        :return: a tuple of the form `(height, width, channels)`
        """
        path = os.path.join(self.images_path, name + '.jpg')
        img = Image.open(path)
        return img.height, img.width, 3

    def ground_truth_mat(self, name):
        """
        Load the ground truth Matlab file identified by the sample name
        (you can get the names from the `train_sample_names`,
        `val_sample_names` and `test_sample_names` attributes)
        :param name: the sample name
        :return: the `groundTruth` entry from the Matlab file
        """
        path = os.path.join(self.gt_path, name + '.mat')
        return self.load_ground_truth_mat(path)

    def segmentations(self, name):
        """
        Load the ground truth segmentations identified by the sample name
        (you can get the names from the `train_sample_names`,
        `val_sample_names` and `test_sample_names` attributes)
        :param name: the sample name
        :return: a list of (H,W) arrays, each of which contains a
        segmentation ground truth
        """
        path = os.path.join(self.gt_path, name + '.mat')
        return self.load_segmentations(path)

    def boundaries(self, name):
        """
        Load the ground truth boundaries identified by the sample name
        (you can get the names from the `train_sample_names`,
        `val_sample_names` and `test_sample_names` attributes)
        :param name: the sample name
        :return: a list of (H,W) arrays, each of which contains a
        boundary ground truth
        """
        path = os.path.join(self.gt_path, name + '.mat')
        return self.load_boundaries(path)

    @staticmethod
    def load_ground_truth_mat(path):
        """
        Load the ground truth Matlab file at the specified path
        and return the `groundTruth` entry.
        :param path: path
        :return: the 'groundTruth' entry from the Matlab file
        """
        gt = scipy.io.loadmat(path)
        return gt['groundTruth']

    @staticmethod
    def load_segmentations(path):
        """
        Load the ground truth segmentations from the Matlab file
        at the specified path.
        :param path: path
        :return: a list of (H,W) arrays, each of which contains a
        segmentation ground truth
        """
        gt = BSDSDataset.load_ground_truth_mat(path)
        num_gts = gt.shape[1]
        return [gt[0,i]['Segmentation'][0,0].astype(np.int32) for i in range(num_gts)]

    @staticmethod
    def load_boundaries(path):
        """
        Load the ground truth boundaries from the Matlab file
        at the specified path.
        :param path: path
        :return: a list of (H,W) arrays, each of which contains a
        boundary ground truth
        """
        gt = BSDSDataset.load_ground_truth_mat(path)
        num_gts = gt.shape[1]
        return [gt[0,i]['Boundaries'][0,0] for i in range(num_gts)]

In [60]:
data = BSDSDataset(bsds_path = ".\\data")

In [64]:
len(data.segmentations('train\\100075'))

6

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt


# Hyper-parameters
input_size = 1
output_size = 1
num_epochs = 60
learning_rate = 0.001

# Toy dataset
x_train = np.array([[3.3], [4.4], [5.5], [6.71], [6.93], [4.168], 
                    [9.779], [6.182], [7.59], [2.167], [7.042], 
                    [10.791], [5.313], [7.997], [3.1]], dtype=np.float32)

y_train = np.array([[1.7], [2.76], [2.09], [3.19], [1.694], [1.573], 
                    [3.366], [2.596], [2.53], [1.221], [2.827], 
                    [3.465], [1.65], [2.904], [1.3]], dtype=np.float32)

# Linear regression model
model = nn.Linear(input_size, output_size)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

# Train the model
for epoch in range(num_epochs):
    # Convert numpy arrays to torch tensors
    inputs = torch.from_numpy(x_train)
    targets = torch.from_numpy(y_train)

    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 5 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# Plot the graph
predicted = model(torch.from_numpy(x_train)).detach().numpy()
plt.plot(x_train, y_train, 'ro', label='Original data')
plt.plot(x_train, predicted, label='Fitted line')
plt.legend()
plt.show()